In [1]:
import os
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import sqlite3
import sys
import re

In [2]:
todaydt = datetime.today()
todaystr = todaydt.strftime("%Y-%m-%d")
e_ticket = f"tickererrors_{todaystr}.txt"
print(f'O arquivo utilizado para verificar os tickers é :{e_ticket}')

tickererrors_2024-04-28.txt


In [3]:
rootdire =  os.getcwd()
rootdire_list = os.listdir(rootdire)

In [4]:
if e_ticket in rootdire_list:
    with open(e_ticket, 'r') as file:
        error_output = file.read()
    pattern = re.compile(r"\['(.*?)'\]")
    tickers_with_errors = pattern.findall(error_output)
    df = pd.DataFrame(tickers_with_errors, columns=['Ticker'])
    df['Ticker'] = df['Ticker'].drop_duplicates()
    df.reset_index(drop=True, inplace=True)
    df.dropna(inplace=True)

In [5]:
conn = sqlite3.connect('yfhistoric.db')
cursor = conn.cursor()
command = "SELECT * FROM tickets"
dfsql = pd.read_sql_query(command, conn)
dfsql.rename(columns={'ticcode': 'Ticker'}, inplace=True)
df = pd.merge(df, dfsql, on='Ticker', how='left', indicator=True)
df.drop(columns={'datatype', '_merge', 'id'}, inplace=True)
conn.close()

In [6]:
df.to_excel(f"output_{todaystr}.xlsx", index=False)